# Model constructor.

> Create and tune pytorch model.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#hide
import torch
import torch.nn as nn

## ResBlock

In [ ]:
#hide
from model_constructor.model_constructor import ResBlock

In [ ]:
block = ResBlock(1, 64, 64)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act_fn): ReLU(inplace=True)
)

In [ ]:
block = ResBlock(4, 64, 64, dw=True)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act_fn): ReLU(inplace=True)
)

In [ ]:
block = ResBlock(4, 64, 64,groups=4)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act_fn): ReLU(inplace=True)
)

In [ ]:
# collapse_output
block = ResBlock(2, 64, 64,act_fn=nn.LeakyReLU(), bn_1st=False)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act_fn): LeakyReLU(negative_slope=0.01)
)

In [ ]:
# collapse_output
lock = ResBlock(2, 32, 64, dw=True)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act_fn): LeakyReLU(negative_slope=0.01)
)

In [ ]:
pool = nn.AvgPool2d(2, ceil_mode=True)

In [ ]:
# collapse_output
block = ResBlock(2, 32, 64, stride=2, dw=True, pool=pool)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (id_conv): Sequential(
    (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (id_conv): ConvBnAct(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_

In [ ]:
from model_constructor.layers import SEModule, SimpleSelfAttention

In [ ]:
# collapse_output
block = ResBlock(2, 32, 64, stride=2, dw=True, pool=pool, se=SEModule)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (se): SEModule(
      (squeeze): AdaptiveAvgPool2d(output_size=1)
      (excitation): Sequential(
        (reduce): Linear(in_features=128, out_features=8, bias=True)
        (se_act): ReLU(inplace=True)
        (expand): Linear(in_features=8, out_features=128, bias=

In [ ]:
# collapse_output
block = ResBlock(2, 32, 64, stride=2, dw=True, pool=pool, se=SEModule, sa=SimpleSelfAttention)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (se): SEModule(
      (squeeze): AdaptiveAvgPool2d(output_size=1)
      (excitation): Sequential(
        (reduce): Linear(in_features=128, out_features=8, bias=True)
        (se_act): ReLU(inplace=True)
        (expand): Linear(in_features=8, out_features=128, bias=

## Stem, Body, Layer, Head

Helper functions to create stem, body and head of model from config.  
Returns `nn.Sequential`.  

In [ ]:
from model_constructor.model_constructor import ModelCfg, make_stem, make_body, make_layer, make_head
from rich import print

In [ ]:
cfg = ModelCfg()
print(cfg.extra_repr())

name: MC
in_chans: 3
num_classes: 1000
block: <class 'model_constructor.model_constructor.ResBlock'>
conv_layer: <class 'model_constructor.layers.ConvBnAct'>
block_sizes: [64, 128, 256, 512]
layers: [2, 2, 2, 2]
norm: <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
act_fn: ReLU(inplace=True)
pool: AvgPool2d(kernel_size=2, stride=2, padding=0)
expansion: 1
groups: 1
dw: False
sa: False
se: False
bn_1st: True
zero_bn: True
stem_stride_on: 0
stem_sizes: [32, 32, 64]
stem_pool: MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
stem_bn_end: False

### Stem

In [ ]:
# collapse_output
stem = make_stem(cfg)
stem

Sequential(
  (conv_0): ConvBnAct(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_1): ConvBnAct(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

### Layer

`make_layer` need `layer_num` argument - number of layer.

In [ ]:
# collapse_output
layer = make_layer(cfg, layer_num=0)
layer

Sequential(
  (bl_0): ResBlock(
    (convs): Sequential(
      (conv_0): ConvBnAct(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act_fn): ReLU(inplace=True)
      )
      (conv_1): ConvBnAct(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (act_fn): ReLU(inplace=True)
  )
  (bl_1): ResBlock(
    (convs): Sequential(
      (conv_0): ConvBnAct(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act_fn): ReLU(inplace=True)
      )
      (conv_1): ConvBnAct(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bia

### Body

`make_body` needs `cfg._make_layer` initialized. As default - `make_layer`.

In [ ]:
# collapse_output
cfg.make_layer = make_layer
body = make_body(cfg)
body

Sequential(
  (l_0): Sequential(
    (bl_0): ResBlock(
      (convs): Sequential(
        (conv_0): ConvBnAct(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_1): ConvBnAct(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (act_fn): ReLU(inplace=True)
    )
    (bl_1): ResBlock(
      (convs): Sequential(
        (conv_0): ConvBnAct(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_1): ConvBnAct(
          (conv): Co

## Head

In [ ]:
# collapse_output
head = make_head(cfg)
head

Sequential(
  (pool): AdaptiveAvgPool2d(output_size=1)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=512, out_features=1000, bias=True)
)

## Model Constructor.

In [ ]:
#hide
from model_constructor import ModelConstructor

In [ ]:
mc  = ModelConstructor()
mc

MC constructor
  in_chans: 3, num_classes: 1000
  expansion: 1, groups: 1, dw: False, div_groups: None
  sa: False, se: False
  stem sizes: [3, 32, 32, 64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [2, 2, 2, 2]

In [ ]:
# collapse_output
mc.stem

Sequential(
  (conv_0): ConvBnAct(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_1): ConvBnAct(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_2): ConvBnAct(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [ ]:
# collapse_output
mc.stem_stride_on = 1
mc.stem

Sequential(
  (conv_0): ConvBnAct(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_1): ConvBnAct(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_2): ConvBnAct(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [ ]:
mc.bn_1st = False

In [ ]:
mc.act_fn = nn.LeakyReLU(inplace=True)

In [ ]:
mc.sa = SimpleSelfAttention
mc.se = SEModule

In [ ]:
# collapse_output
mc.body.l_0

Sequential(
  (bl_0): ResBlock(
    (convs): Sequential(
      (conv_0): ConvBnAct(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_1): ConvBnAct(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SEModule(
        (squeeze): AdaptiveAvgPool2d(output_size=1)
        (excitation): Sequential(
          (reduce): Linear(in_features=64, out_features=4, bias=True)
          (se_act): ReLU(inplace=True)
          (expand): Linear(in_features=4, out_features=64, bias=True)
          (se_gate): Sigmoid()
        )
      )
    )
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (bl_1): ResBlock(
    (conv

model_constructor
by ayasyrev